In [ ]:
import os
import cv2
import numpy as np
import shutil
from tqdm import tqdm

In [ ]:
def find_unique_grayscale_values(masks_folder):
    """Utility to print unique grayscale values from a mask dataset"""
    unique_values = set()
    
    for mask_file in os.listdir(masks_folder):
        if mask_file.endswith(".png"):
            mask_path = os.path.join(masks_folder, mask_file)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            unique_values.update(np.unique(mask))
    
    print("Unique grayscale values found:", sorted(unique_values))

In [ ]:
mask_folder = "path/to/your/training_masks"
find_unique_grayscale_values(mask_folder)

Unique grayscale values found: [0, 64, 128, 191, 255]


In [ ]:
def create_yolo_annotations(images_folder, masks_folder, output_folder, grayscale_to_class, images_output_folder):
    """Convert grayscale segmentation masks to YOLO-format bounding boxes"""
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(images_output_folder, exist_ok=True)
    
    image_files = [f for f in os.listdir(images_folder) if f.endswith(".png")]
    
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(images_folder, image_file)
        mask_path = os.path.join(masks_folder, image_file)
        
        if not os.path.exists(mask_path):
            print(f"Mask not found for {image_file}, skipping...")
            continue
        
        # Copy image to new folder
        shutil.copy(image_path, os.path.join(images_output_folder, image_file))
        
        # Read mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        h, w = mask.shape
        
        # Find unique grayscale values (excluding 0, which is the background)
        unique_values = np.unique(mask)
        unique_values = unique_values[unique_values > 0]
        
        annotations = []
        for value in unique_values:
            if value not in grayscale_to_class:
                continue
            class_id = grayscale_to_class[value]
            
            # Create a binary mask for the current object
            obj_mask = (mask == value).astype(np.uint8) * 255
            
            # Find contours
            contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                x, y, width, height = cv2.boundingRect(contour)
                
                # Convert to YOLO format
                x_center = (x + width / 2) / w
                y_center = (y + height / 2) / h
                norm_width = width / w
                norm_height = height / h
                
                annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")
        
        # Save annotation file
        annotation_path = os.path.join(output_folder, image_file.replace(".png", ".txt"))
        with open(annotation_path, "w") as f:
            f.write("\n".join(annotations))

In [ ]:
# Set paths
train_images = "path/to/training_images"
train_masks = "path/to/training_masks"
train_labels_output = "path/to/output/labels/train"
train_images_output = "path/to/output/images/train"

val_images = "path/to/validation_images"
val_masks = "path/to/validation_masks"
val_labels_output = "path/to/output/labels/val"
val_images_output = "path/to/output/images/val"

In [6]:
# Define grayscale-to-class mapping 
grayscale_to_class = {
    64: 0,   # Class 0
    128: 1,  # Class 1
    191: 2,  # Class 2
    255: 3   # Class 3
}

In [9]:
# Run annotation creation
create_yolo_annotations(train_images, train_masks, train_labels_output, grayscale_to_class, train_images_output)
create_yolo_annotations(val_images, val_masks, val_labels_output, grayscale_to_class, val_images_output)


Processing images:   0%|          | 0/45 [00:00<?, ?it/s]

Processing images: 100%|██████████| 11/11 [00:00<00:00, 97.51it/s]


## Visualizing the YOLO bounding boxes 
It is a good way to verify if your annotation logic is working correctly. 

In [ ]:
def visualize_yolo_annotations(image_path, annotation_path, output_path):
    """Visualize YOLO bounding boxes over the original image"""
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image:", image_path)
        return
    
    h, w, _ = image.shape

    if not os.path.exists(annotation_path):
        print("Annotation file not found:", annotation_path)
        return

    with open(annotation_path, "r") as f:
        lines = f.read().splitlines()
        
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center, y_center, box_width, box_height = map(float, parts)

        # Convert from YOLO format to pixel coordinates
        x_center *= w
        y_center *= h
        box_width *= w
        box_height *= h

        x1 = int(x_center - box_width / 2)
        y1 = int(y_center - box_height / 2)
        x2 = int(x_center + box_width / 2)
        y2 = int(y_center + box_height / 2)

        # Draw rectangle and class label
        color = (0, 255, 0)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"Class {int(class_id)}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imwrite(output_path, image)
    print(f"Saved visualized image to: {output_path}")

# Visualize a YOLO annotation
image_path = "path/to/output/images/train/Tile1.png"
annotation_path = "path/to/output/labels/train/Tile1.txt"
output_path = "path/to/visual_output/Tile1_with_boxes.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

visualize_yolo_annotations(image_path, annotation_path, output_path)

Saved visualized image to: /home/usuaris/imatge/sara.alonso.del.hoyo/TFG_BAL/data/Dataset_Alejandro/debug/Tile1_with_boxes.png
